In [2]:
import numpy as np
import math

def flujo_canny_manual(roi, nombre_ejercicio, vec_mags, umbrales):
    """
    Simula el flujo completo de Canny para un ROI de 3x3.
    Args:
        roi: Matriz 3x3 de entrada.
        nombre_ejercicio: Nombre para imprimir.
        vec_mags: Tupla con magnitudes supuestas de los dos vecinos a comparar en NMS.
        umbrales: Tupla (Bajo, Alto) para Histéresis.
    """
    print(f"\n{'='*50}")
    print(f"PROCESANDO: {nombre_ejercicio}")
    print(f"{'='*50}")
    print("Matriz Original:\n", roi)

    # ---------------------------------------------------------
    # ETAPA 1: SUAVIZADO GAUSSIANO (Aproximación 3x3)
    # ---------------------------------------------------------
    kernel_gauss = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16.0
    val_suavizado = np.sum(roi * kernel_gauss)
    
    print(f"\n--- ETAPA 1: Suavizado ---")
    print(f"Cálculo: Suma ponderada del kernel gaussiano.")
    print(f"Valor del píxel central suavizado: {val_suavizado:.2f}")
    
    # NOTA: Para coincidir con tus cálculos manuales de Sobel, 
    # usaremos la ROI original en el paso 2, no la suavizada.
    
    # ---------------------------------------------------------
    # ETAPA 2: GRADIENTE (SOBEL)
    # ---------------------------------------------------------
    kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    ky = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    
    Gx = np.sum(roi * kx)
    Gy = np.sum(roi * ky)
    
    magnitud = math.sqrt(Gx**2 + Gy**2)
    angulo_rad = math.atan2(Gy, Gx)
    angulo_deg = math.degrees(angulo_rad)
    
    # Conversión del Ingeniero (Regla 180)
    angulo_final = angulo_deg
    if angulo_final < 0:
        angulo_final += 180
    elif angulo_final >= 180:
        angulo_final -= 180
        
    print(f"\n--- ETAPA 2: Gradientes ---")
    print(f"Gx: {Gx} | Gy: {Gy}")
    print(f"Magnitud: {magnitud:.2f}")
    print(f"Ángulo calculado: {angulo_deg:.2f}°")
    print(f"Ángulo normalizado (0-180): {angulo_final:.2f}°")

    # ---------------------------------------------------------
    # ETAPA 3: SUPRESIÓN DE NO-MÁXIMOS (NMS)
    # ---------------------------------------------------------
    print(f"\n--- ETAPA 3: Supresión de No-Máximos ---")
    
    # Determinar dirección
    direccion = ""
    if (0 <= angulo_final <= 22.5) or (157.5 <= angulo_final < 180):
        direccion = "Horizontal (Borde Vertical)"
    elif 67.5 <= angulo_final <= 112.5:
        direccion = "Vertical (Borde Horizontal)"
    elif 22.5 < angulo_final < 67.5:
        direccion = "Diagonal 1 (NE-SO /)"
    elif 112.5 < angulo_final < 157.5:
        direccion = "Diagonal 2 (NO-SE \)"
        
    print(f"Dirección detectada: {direccion}")
    print(f"Comparando centro ({magnitud:.2f}) con vecinos supuestos: {vec_mags}")
    
    es_maximo = False
    if magnitud >= vec_mags[0] and magnitud >= vec_mags[1]:
        es_maximo = True
        print("RESULTADO NMS: ¡El píxel SE MANTIENE! (Es un máximo local)")
    else:
        print("RESULTADO NMS: El píxel se suprime a 0.")

    # ---------------------------------------------------------
    # ETAPA 4: UMBRALIZACIÓN POR HISTÉRESIS
    # ---------------------------------------------------------
    if es_maximo:
        print(f"\n--- ETAPA 4: Histéresis ---")
        T_L, T_H = umbrales
        tipo_borde = "Nada"
        if magnitud >= T_H:
            tipo_borde = "BORDE FUERTE (Definitivo)"
        elif magnitud >= T_L:
            tipo_borde = "BORDE DÉBIL (Pendiente de conexión)"
        else:
            tipo_borde = "NO ES BORDE (Menor a T_L)"
        print(f"Umbrales: Bajo={T_L}, Alto={T_H}")
        print(f"Decisión Final: {tipo_borde}")


# =================================================================
# DATOS DE LOS 3 EJERCICIOS
# =================================================================

# EJERCICIO 1: BORDE VERTICAL
roi_1 = np.array([[10, 10, 200], [10, 10, 200], [10, 10, 200]])
# En el chat dijimos: Vecino Izq=0, Vecino Der=760
flujo_canny_manual(roi_1, "EJERCICIO 1 (Vertical)", vec_mags=(0, 760), umbrales=(200, 500))

# EJERCICIO 2: BORDE HORIZONTAL
roi_2 = np.array([[200, 200, 200], [200, 200, 200], [50, 50, 50]])
# En el chat dijimos: Vecino Arriba=0, Vecino Abajo=600
flujo_canny_manual(roi_2, "EJERCICIO 2 (Horizontal)", vec_mags=(0, 600), umbrales=(200, 500))

# EJERCICIO 3: BORDE DIAGONAL
roi_3 = np.array([[50, 255, 255], [0, 50, 255], [0, 0, 50]])
# En el chat dijimos: Vecinos diagonales aprox 200
flujo_canny_manual(roi_3, "EJERCICIO 3 (Diagonal)", vec_mags=(200, 200), umbrales=(200, 500))


PROCESANDO: EJERCICIO 1 (Vertical)
Matriz Original:
 [[ 10  10 200]
 [ 10  10 200]
 [ 10  10 200]]

--- ETAPA 1: Suavizado ---
Cálculo: Suma ponderada del kernel gaussiano.
Valor del píxel central suavizado: 57.50

--- ETAPA 2: Gradientes ---
Gx: 760 | Gy: 0
Magnitud: 760.00
Ángulo calculado: 0.00°
Ángulo normalizado (0-180): 0.00°

--- ETAPA 3: Supresión de No-Máximos ---
Dirección detectada: Horizontal (Borde Vertical)
Comparando centro (760.00) con vecinos supuestos: (0, 760)
RESULTADO NMS: ¡El píxel SE MANTIENE! (Es un máximo local)

--- ETAPA 4: Histéresis ---
Umbrales: Bajo=200, Alto=500
Decisión Final: BORDE FUERTE (Definitivo)

PROCESANDO: EJERCICIO 2 (Horizontal)
Matriz Original:
 [[200 200 200]
 [200 200 200]
 [ 50  50  50]]

--- ETAPA 1: Suavizado ---
Cálculo: Suma ponderada del kernel gaussiano.
Valor del píxel central suavizado: 162.50

--- ETAPA 2: Gradientes ---
Gx: 0 | Gy: -600
Magnitud: 600.00
Ángulo calculado: -90.00°
Ángulo normalizado (0-180): 90.00°

--- ETAPA 3: 

<>:71: SyntaxWarning: invalid escape sequence '\)'
<>:71: SyntaxWarning: invalid escape sequence '\)'
/tmp/ipykernel_121865/2735807926.py:71: SyntaxWarning: invalid escape sequence '\)'
  direccion = "Diagonal 2 (NO-SE \)"
